In [1]:
# using python to perform ETL

In [2]:
%%python -m pip install pymongo
!pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement pymongo (from versions: none)


ERROR: No matching distribution found for pymongo


CalledProcessError: Command 'b'!pip install sqlalchemy\n'' returned non-zero exit status 1.

ERROR: Could not find a version that satisfies the requirement pymongo (from versions: none)


ERROR: No matching distribution found for pymongo


CalledProcessError: Command 'b'!pip install sqlalchemy\n'' returned non-zero exit status 1.

In [ ]:
import os
import json
import numpy
import datetime
import certifi
import pandas as pd

import pymongo
import sqlalchemy
from sqlalchemy import create_engine, text

In [ ]:
print(f"Running SQL Alchemy Version: {sqlalchemy.__version__}")
print(f"Running PyMongo Version: {pymongo.__version__}")

In [ ]:
mysql_args = {
    "uid" : "root",
    "pwd" : "Wishbone1",
    "hostname" : "localhost",
    "dbname" : "grocery_inventory_db"
}

# The 'cluster_location' must either be "atlas" or "local".
mongodb_args = {
    "user_name" : "dorothy_philip",
    "password" : "Wishbone1",
    "cluster_name" : "Cluster0",
    "cluster_subnet" : "xxxxx",
    "cluster_location" : "local", # "local"
    "db_name" : "grocery_inventory_db"
}

In [ ]:
# Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Wishbone1"

src_dbname = "classicmodels"
dst_dbname = "classicmodels_dw"

In [ ]:
# Fetch grocery inventory data from MySQL
sql_query = "SELECT * FROM grocery_inventory;"
grocery_df = get_sql_dataframe(sql_query, 
                               uid="your_username", 
                               pwd="your_password", 
                               hostname="your_host", 
                               dbname="grocery_inventory_db")

# Get MongoDB client (Atlas or local)
mongo_client = get_mongo_client(cluster_location="local")  

# Specify database and collection name
db_name = "grocery_db"
collection_name = "grocery_inventory"

# Convert DataFrame to dictionary for MongoDB
grocery_data = grocery_df.to_dict(orient="records")

# Insert data into MongoDB
db = mongo_client[db_name]
collection = db[collection_name]
collection.insert_many(grocery_data)

print("Grocery inventory data successfully inserted into MongoDB!")

# Fetch data from MongoDB to verify insertion
query = {}  # Empty query to get all documents
mongo_df = get_mongo_dataframe(mongo_client, db_name, collection_name, query)

# Display the first 5 rows
print(mongo_df.head())



In [ ]:
from sqlalchemy import create_engine, text

# MySQL connection details
mysql_args = {
    "user": "your_user",
    "password": "your_password",
    "host": "localhost",
    "dbname": "grocery_inventory_db"
}

# Establish connection
conn_str = f"mysql+pymysql://{mysql_args['user']}:{mysql_args['password']}@{mysql_args['host']}/{mysql_args['dbname']}"
sqlEngine = create_engine(conn_str)
connection = sqlEngine.connect()

# Read and execute SQL schema from file
sql_file_path = "DS2002_DimDateProduct.sql"
with open(sql_file_path, "r") as file:
    sql_script = file.read()

# Execute SQL schema
try:
    connection.execute(text(sql_script))
    print("Database schema executed successfully.")
except Exception as e:
    print(f"Error executing schema: {e}")

# Close connection
connection.close()


In [ ]:
# Define CSV File Path and Load into MySQL
data_dir = os.getcwd()
data_file = os.path.join(data_dir, "Cleaned_Grocery_Inventory.csv")

df = pd.read_csv(data_file)
# Read CSV into Pandas DataFrame
df = pd.read_csv(data_file)

# MySQL Connection
mysql_args = {
    "user": "your_user",
    "password": "your_password",
    "host": "localhost",
    "dbname": "grocery_inventory_db"
}
sqlEngine = create_engine(f"mysql+pymysql://{mysql_args['user']}:{mysql_args['password']}@{mysql_args['host']}/{mysql_args['dbname']}")

table_name = "grocery_inventory"
df.to_sql(name=table_name, con=sqlEngine, if_exists="replace", index=False)
print(f"Data successfully inserted into `{table_name}` table in `{mysql_args['dbname']}` database.")

# Load Data into MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["dim_date_product_db"]

In [ ]:
# Extract Data from MongoDB
db = client["dim_date_product_db"]
query = {}  # Select all records
collection = "dim_date_product"
df_dim_date_product = get_mongo_dataframe(client, "dim_date_product_db", collection, query)
df_dim_date_product.head(2)


In [ ]:
# At this point, run the [DS2002 SQL file]
sql_dim_date = "SELECT date_key, full_date FROM grocery_inventory_db.dim_date;"
df_dim_date = get_sql_dataframe(sql_dim_date, **mysql_args)
df_dim_date.full_date = df_dim_date.full_date.astype('datetime64[ns]').dt.date
df_dim_date.head(10)


In [ ]:
# Look up the Surrogate Primary Key (date_key) that corresponds to the groceryDate column
df_dim_groceryDate = df_dim_date.rename(columns={"date_key": "groceryDate_key", "full_date": "groceryDate"})
df_grocery_inventory.groceryDate = df_grocery_inventory.groceryDate.astype('datetime64[ns]').dt.date
df_grocery_inventory = pd.merge(df_grocery_inventory, df_dim_groceryDate, on='groceryDate', how='left')
df_grocery_inventory.drop(['groceryDate'], axis=1, inplace=True)
df_grocery_inventory.head(2)

In [ ]:
# Load the transformed grocery_inventory DataFrame into the New Data Warehouse
def set_dataframe(df, table_name, pk_column, db_operation, **args):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{args['uid']}:{args['pwd']}@{args['hostname']}/{args['dbname']}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()

    print(f"Inserting into {table_name} with primary key {pk_column}")

    # Insert or replace data
    if db_operation == "insert":
        try:
            df.to_sql(table_name, con=connection, index=False, if_exists='append')
            print("Data inserted successfully.")
        except Exception as e:
            print(f"Error inserting data into {table_name}: {e}")

        try:
            result = connection.execute(f"SHOW COLUMNS FROM {table_name} LIKE '{pk_column}';")
            if not result.fetchone():
                print(f"Adding primary key column {pk_column} as AUTO_INCREMENT.")
                connection.execute(f"ALTER TABLE {table_name} ADD COLUMN {pk_column} INT AUTO_INCREMENT PRIMARY KEY;")
        except Exception as e:
            print(f"Error adding primary key to {table_name}: {e}")

    elif db_operation == "replace":
        try:
            df.to_sql(table_name, con=connection, index=False, if_exists='replace')
            connection.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            print("Data replaced successfully.")
        except Exception as e:
            print(f"Error replacing data into {table_name}: {e}")

    connection.close()


In [ ]:
# fetching data for dimensions table

In [ ]:
# Fetching data from grocery inventory
sql_grocery = "SELECT * FROM grocery_inventory_db.grocery_inventory;"
df_grocery = get_dataframe(user_id, pwd, host_name, src_dbname, sql_grocery)
df_grocery.head(2)

In [ ]:
# Fetching data from grocery products
sql_products = "SELECT * FROM grocery_inventory_db.products;"
df_products = get_dataframe(user_id, pwd, host_name, src_dbname, sql_products)
df_products.head(2)

In [ ]:
# Perform any necessary transformations

In [ ]:
# GROCERY INVENTORY

# 1. Remove unnecessary columns
drop_cols = ['supplier_name', 'supplier_contact', 'location', 'expiration_date']
df_grocery.drop(drop_cols, axis=1, inplace=True)

# 2. Rename "item_id" to "grocery_id" for consistency
df_grocery.rename(columns={"item_id": "grocery_id"}, inplace=True)

# 3. Insert a new primary key column
df_grocery.insert(0, "grocery_key", range(1, df_grocery.shape[0] + 1))

# 4. Display first 2 rows
df_grocery.head(2)


In [ ]:
# GROCERY PRODUCTS

# 1. Remove unnecessary columns
drop_cols = ['brand', 'manufacturer', 'size', 'weight']
df_products.drop(drop_cols, axis=1, inplace=True)

# 2. Rename "product_code" to "product_id"
df_products.rename(columns={"product_code": "product_id"}, inplace=True)

# 3. Insert a new primary key column
df_products.insert(0, "product_key", range(1, df_products.shape[0] + 1))

# 4. Display first 2 rows
df_products.head(2)


In [ ]:
# Show all tables in the database
sql_check_table = "SHOW TABLES;"
df_tables = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_check_table)
print(df_tables)


In [ ]:
# load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

In [ ]:
db_operation = "update"

tables = [('dim_grocery', df_grocery, 'grocery_key'),
          ('dim_products', df_products, 'product_key')]


In [ ]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(dataframe, table_name, primary_key, db_operation, **mysql_args)


In [ ]:
# creating and populating the fact table called sales_fact

In [ ]:
# Show all tables in the database
sql_check_table = "SHOW TABLES;"
df_tables = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_check_table)
print(df_tables)


In [ ]:
sql_sales_fact = """
SELECT 
    g.grocery_key, 
    py.payment_key, 
    py.amount AS total_price,
    py.paymentDate_key
FROM dim_payments py
JOIN dim_grocery g ON py.item_id = g.grocery_key
"""

df_sales_fact = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_sales_fact)
print(df_sales_fact.shape)  # Ensure data exists
print(df_sales_fact.head(2))  # Check first 2 rows



In [ ]:
# Insert a new primary key column
df_sales_fact.insert(0, "sales_fact_key", range(1, df_sales_fact.shape[0] + 1))

# Display first 2 rows
df_sales_fact.head(2)


In [ ]:
## Insert data into the sales_fact table
df_sales_fact.to_sql('sales_fact', con=sqlEngine, if_exists='replace', index=False)

print("Data successfully inserted into the sales_fact table.")
df_sales_fact.head(5)


In [ ]:
# Ensure that the new data warehouse contains the correct data
sql_check_dw = "SHOW TABLES;"
df_dw_tables = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_check_dw)
print(df_dw_tables)

In [ ]:
# Retrieve total sales data for each grocery item
sql_query = """
SELECT 
    g.grocery_key, 
    g.item_name, 
    COUNT(sf.sales_fact_key) AS total_sales, 
    SUM(py.amount) AS total_revenue, 
    AVG(py.amount) AS avg_payment_amount
FROM sales_fact sf
JOIN dim_payments py ON sf.payment_key = py.payment_key
JOIN dim_grocery g ON py.item_id = g.grocery_key
GROUP BY g.grocery_key, g.item_name
ORDER BY total_revenue DESC;
"""

# Execute the SQL query and load results into Pandas DataFrame
df_total_sales_by_item = pd.read_sql(sql_query, sqlEngine)

# Display first 2 rows
print(df_total_sales_by_item.head(2))
